In [1]:
# To run this you need to have the 'genderbias' package installed
# Since the package is not on PyPI, you will need to run 'pip -e .' from the cloned git repository to do so

# Initially just import the Document class to load texts
from genderbias import Document

In [2]:
# Document can load text inline, as shown here
inline_example = Document("I thoroughly recommend PERSON due to their extreme hate in this endeavour of love.")

In [3]:
# Documents have useful methods, eg. the words:
inline_example.words()

['I',
 'thoroughly',
 'recommend',
 'PERSON',
 'due',
 'to',
 'their',
 'extreme',
 'hate',
 'in',
 'this',
 'endeavour',
 'of',
 'love',
 '.']

In [4]:
# Also useful is words_with_indices, for finding where a word is in a text, eg. to highlight it:
inline_example.words_with_indices()

[('I', 0, 1),
 ('thoroughly', 2, 12),
 ('recommend', 13, 22),
 ('PERSON', 23, 29),
 ('due', 30, 33),
 ('to', 34, 36),
 ('their', 37, 42),
 ('extreme', 43, 50),
 ('hate', 51, 55),
 ('in', 56, 58),
 ('this', 59, 63),
 ('endeavour', 64, 73),
 ('of', 74, 76),
 ('love', 77, 81),
 ('.', 81, 82)]

In [5]:
# A Detector needs one method: get_report; this Detector reports how many words, using the first method above
# This requires the Report class, since get_report methods are expected to generate one
from genderbias.detector import Report

class WordsDetector:
    def get_report(self, document):
        report = Report("Word Count")
        report.set_summary("The document has {} words".format(len(document.words())))
        return report

words_report = WordsDetector().get_report(inline_example)
print(words_report)

Word Count
 SUMMARY: The document has 15 words


In [6]:
# Here's another detector, just looking for one particular word
# We need another import here, since WordFlaggingDetector marks words using Flags and Issues
from genderbias.detector import Flag, Issue

class WordFlaggingDetector:
    def get_report(self, document):
        report = Report("Word Flagger")
        my_word = 'recommend'
        if my_word in document.words():
            for word, start, stop in document.words_with_indices():
                if my_word == word:
                    report.add_flag(Flag(start, stop,
                                         Issue("Word Flagger", "Found '{}'".format(my_word))))
        return report
    
flagging_report = WordFlaggingDetector().get_report(inline_example)
print(flagging_report)

Word Flagger
 [13-22]: Word Flagger: Found 'recommend'
 SUMMARY: [None available]


In [7]:
# The last Detector here, just looking for love and hate.
# This uses the 'bias' property of an Issue, which defaults to Issue.negative_result (cf Issue.positive_result)
# Note how the positive result is shown only in the raw JSON/dict-based output
class LoveHateDetector:
    def get_report(self, document):
        report = Report("Love & Hate")
        if 'love' in document.words():
            for word, start, stop in document.words_with_indices():
                if 'love' == word:
                    report.add_flag(Flag(start, stop,
                                         Issue("Love & Hate", "Found love :)", bias=Issue.positive_result)))
        if 'hate' in document.words():
            for word, start, stop in document.words_with_indices():
                if 'hate' == word:
                    report.add_flag(Flag(start, stop,
                                         Issue("Love & Hate", "Found hate :(", bias=Issue.negative_result)))

        return report
    
flagging_report = LoveHateDetector().get_report(inline_example)
print(flagging_report)
print(flagging_report.to_dict())

Love & Hate
 [51-55]: Love & Hate: Found hate :(
 SUMMARY: [None available]
{'name': 'Love & Hate', 'summary': '', 'flags': [(77, 81, 'Love & Hate', 'Found love :)', '', 1.0), (51, 55, 'Love & Hate', 'Found hate :(', '', -1.0)]}
